In [96]:
import pandas as pd
import numpy as np
import math
import json
import yaml
import operator

from itertools import combinations
from collections import Counter as ctr

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from tqdm import tqdm
import ast

In [14]:
index_food_tfidf = pd.read_csv('new_food_tfidfindex.csv', names=['word', 'tf-idf'])
index_food_tfidf["tf-idf"] = [ast.literal_eval(x) for x in index_food_tfidf['tf-idf']]

In [15]:
index_food_tfidf

,word,tf-idf
0,alwaysalway,{94736: 5.940502063907854}
1,venegar,{84735: 5.940502063907854}
2,flaver,"{83749: 5.412181230334135, 107751: 5.412181230..."
3,yasner,{112034: 8.91075309586178}
4,outlandish,{83793: 8.91075309586178}
5,claxton,{172856: 5.940502063907854}
6,mishmong,{115018: 8.91075309586178}
7,podi,{68916: 4.45537654793089}
8,flourlesss,{51060: 8.91075309586178}
9,carpatho,{187454: 8.91075309586178}


In [92]:
data_combined = pd.read_csv('new_recipes.csv')

In [90]:
rt = index_food_tfidf.loc[index_food_tfidf.word == 'chocol', 'tf-idf'].item()
rt2 = index_food_tfidf.loc[index_food_tfidf.word == 'churro', 'tf-idf'].item()
p = set(rt.keys()).intersection(rt2.keys())

In [91]:
p

{49568, 49569, 53095, 53099, 53100, 53102, 53104, 54921, 130654, 193898}

In [89]:
x = clean('churro')
x

['churro']

In [108]:
def snippet (doc_id, q):
    q = clean(q)
    sent_doc = pd.DataFrame()
    cosine_df = pd.DataFrame()
    
    #Grab Full doc from corpus, since I dont keep periods
    doc = data_combined.loc[data_combined.index == doc_id, 'content'].item()
    
    #Tokenize into sentences and clean
    sent_doc['sent'] = sent_tokenize(doc)
    sent_doc['clean'] = [clean(x) for x in sent_doc.sent]
    
    #Create vector from query and sentence
    cosine = []
    for x in sent_doc.clean:
        x_set = set(x)
        q_set = set(q)
        vector = x_set.union(q_set)
    
    #Calculate vectore values for both
        q_v = []
        s_v = []
        for w in vector:
            if w in q_set:
                q_v_temp = index_food_tfidf.loc[index_food_tfidf.word == w]['tf-idf'].item().get(doc_id)
                if(q_v_temp == None):
                    q_v.append(0)
                else:
                    q_v.append(q_v_temp)
            else:
                q_v.append(0)
            if w in x_set:
                s_v_temp = index_food_tfidf.loc[index_food_tfidf.word == w]['tf-idf'].item().get(doc_id)
                if(s_v_temp == None):
                    s_v.append(0)
                else:
                    s_v.append(s_v_temp)
            else:
                s_v.append(0)
    
    #Calcumate cosine simularity
        c = 0
        for i in range(len(vector)):
            c += q_v[i] * s_v[i]
        cosine.append(c / math.sqrt((math.pow(sum(q_v), 2)) * (math.pow(sum(s_v), 2))))
    cosine_df['sim'] = cosine
    cosine_df = cosine_df.sort_values(by=['sim'], ascending =False).reset_index()
    sent_pos = list(cosine_df[0:3]['index'])
    

    snip = []
    snip.append(data_combined.name[doc_id])
    for x in sent_pos:
        snip.append(sent_doc.sent[x])
    return snip

In [53]:
lemmatizer = WordNetLemmatizer()
stops = stopwords.words('english')
coll_stops = ["prepare", "course", "dietary", "easy", "pepper", "recipe", "salt"]
stops.extend(coll_stops)
ps = PorterStemmer()

stops.extend(coll_stops)

def clean(doc):
    doc_low = doc.lower().replace("–", " ")
    words = word_tokenize(doc_low)
    words = [lemmatizer.lemmatize(w).strip() for w in words if not w in stops and w.isalpha()]
    words = [ps.stem(w) for w in words]
    return words

In [103]:
def get_candidate_resources(query):
    clean_query = clean(query)
    CR = []
    word_key_list = []
    if len(clean_query) == 1:
        only_word = clean_query[0]    #sw = single word
        sw_dict = index_food_tfidf.loc[index_food_tfidf.word == only_word, 'tf-idf'].item()
        CR.extend(list(sw_dict.keys()))
        if len(CR) > 50:
            return CR[:50]
        else: return CR
    for w in clean_query:
        w_dict = index_food_tfidf.loc[index_food_tfidf.word == w, 'tf-idf'].item()
        word_key_list.append(list(w_dict.keys()))
    CR.extend(set(word_key_list[0]).intersection(*word_key_list[1:]))
    if len(clean_query) == 2:
        if len(CR)<50:
            first_word = clean_query[0]
            second_word = clean_query[1]
            first_dict = index_food_tfidf.loc[index_food_tfidf.word == first_word, 'tf-idf'].item()
            second_dict = index_food_tfidf.loc[index_food_tfidf.word == second_word, 'tf-idf'].item()
            CR.extend(list(first_dict.keys()))
            CR.extend(list(second_dict.keys()))
            if len(set(CR))>50:
                return CR[:50]
            else: return set(CR)
        else:
            if len(CR)>50: return CR[:50]
            else: return CR
    elif len(CR)<50:     #if the list of candidate resources is less than 50
        combs = combinations(clean_query, len(query)-1)     #use n-1 terms from the query
        for comb in list(combs):        #for combination in combinations
            word_key_list = []
            for w in list(comb):        #for word in n-1 combination
                w_dict = index_food_tfidf.loc[index_food_tfidf.word == w, 'tf-idf'].item()
                word_key_list.append(list(w_dict.keys()))
            CR.extend(set(word_key_list[0]).intersection(*word_key_list[1:])) #find the intersection between all n-1 query terms
            CR = set(CR)
            if len(set(CR))<50:
                continue
            elif len(set(CR))>50:
                CR = set(CR)[:49]
                break
            else: break
    return CR[:50]

In [102]:
def relevance_ranking(q, cand_resources):
    clean_query = clean(q)
    rel_docs = {}
    rel_score = 0
    if len(clean_query)==1:
        w = clean_query[0]
        tfidf_dict = index_food_tfidf.loc[index_food_tfidf.word == w, 'tf-idf'].item()
        for d in cand_resources:
            result = tfidf_dict[d]
            rel_docs[d] = result
    else:
        for d in cand_resources:
            for word in clean_query:
                tfidf_dict = index_food_tfidf.loc[index_food_tfidf.word == word, 'tf-idf'].item()
                if tfidf_dict.get(d):
                    result = tfidf_dict[d]
                    rel_score += result
            rel_docs[d] = rel_score
    sorted_rel_docs = sorted(rel_docs.items(), key=operator.itemgetter(1), reverse=True)
    results = [i[0] for i in sorted_rel_docs[:5]]
    return results

In [150]:
q = "turducken"
r = get_candidate_resources(q)

In [151]:
len(r)

4

In [152]:
ranked_resources = relevance_ranking(q, r)

In [153]:
ranked_resources

[219678, 217771, 217772, 212645]

In [157]:
q = 'turducken'
snippet(219678, q)

['umami stuffing',
 'the rich earthiness of this dish is a nice counterpart to sweeter aspects of a thanksgiving meal as well as a pleasant support for the meat, be it ham or turkey, or even tofurkey or turducken.',
 'time-to-make course preparation occasion stuffings-dressings side-dishes holiday-event thanksgiving taste-mood savory 4-hours-or-less a recipe developed on the fly for thanksgiving 2008 when there was a disagreement about how the stuffing should be prepared; i wanted savory, everyone else wanted sweet.',
 'at the end of the day, there was less of mine to put away.']

In [158]:
snippet(217771, q)

['turducken  a chicken in a duck in a turkey',
 'course main-ingredient cuisine preparation occasion north-american for-large-groups very-low-carbs main-dish poultry oven dinner-party holiday-event chicken turkey dietary christmas thanksgiving high-protein low-carb high-in-something low-in-something meat duck whole-turkey equipment number-of-servings a turducken is a de-boned turkey stuffed with a de-boned duck, which itself is stuffed with a small de-boned chicken.',
 'the name is a portmanteau of those ingredients, turkey, duck, and chicken.',
 'the birds can be deboned the day before to save some time since this recipe requires a long slow cook time.']

In [159]:
snippet(217772, q)

['turducken roulade',
 'time-to-make course main-ingredient cuisine preparation occasion north-american main-dish poultry american cajun southern-united-states holiday-event thanksgiving meat 4-hours-or-less this is a roulade version of the cajun specialty turducken.',
 'i am also posting recipes for the cajun cornbread stuffing and the cajun rice dressing used in this recipe.',
 'boneless turkey breast boneless duck breast boneless chicken breast cornbread stuffing mix rice dressing bacon']

In [160]:
snippet(212645, q)

['the turducken of thanksgiving sides',
 'now your turducken thanksgiving just got real!',
 '60-minutes-or-less time-to-make course main-ingredient preparation occasion yams-sweet-potatoes low-protein stuffings-dressings side-dishes potatoes vegetables holiday-event dietary thanksgiving low-in-something mashed-potatoes mashed sweet potatoes with pecans centered by perfectly mashed potatoes in a stuffing crust.',
 'this side dish is easy to put together and simply delicious!']

In [156]:
#snippet(41320, q)